## ДЗ 2

### Задание 1.
Задание: обучите три классификатора:

1) на токенах с высокой частотой

2) на токенах со средней частотой

3) на токенах с низкой частотой

Сравните полученные результаты, оцените какие токены наиболее важные для классификации.
#### Сделано

### Задание 2.
найти фичи с наибольшей значимостью, и вывести их
#### Сделано

### Задание 3.
1) сравнить count/tf-idf/hashing векторайзеры/полносвязанную сетку (построить classification_report)
#### Сделано кроме полносвязной сетки (почему-то валится с ошибкой преобразования)

2) подобрать оптимальный размер для hashing векторайзера
#### Сделано

3) убедиться что для сетки нет переобучения


In [61]:
import pandas as pd
import numpy as np
from sklearn.metrics import *
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
import nltk
import re

In [63]:
with open('hw2_corpus.txt', 'r') as f:
    df = pd.DataFrame({'category': t[0], 'text': t[1]} for t in (l.split(' ', 1) for l in f.readlines()))
df['category'] = df['category'].apply(lambda t: re.search(r'\d+', t).group())
df

,category,text
0,2,Stuning even for the non-gamer: This sound tra...
1,2,The best soundtrack ever to anything.: I'm rea...
2,2,Amazing!: This soundtrack is my favorite music...
3,2,Excellent Soundtrack: I truly like this soundt...
4,2,"Remember, Pull Your Jaw Off The Floor After He..."
...,...,...
9995,2,A revelation of life in small town America in ...
9996,2,Great biography of a very interesting journali...
9997,1,Interesting Subject; Poor Presentation: You'd ...
9998,1,Don't buy: The box looked used and it is obvio...


In [64]:
from string import punctuation
stopwords = nltk.corpus.stopwords.words('english')
noise = set(stopwords + list(punctuation))
df['text_tokens'] = df['text'].apply(lambda t: [w for w in nltk.word_tokenize(t) if w not in noise])
df

,category,text,text_tokens
0,2,Stuning even for the non-gamer: This sound tra...,"[Stuning, even, non-gamer, This, sound, track,..."
1,2,The best soundtrack ever to anything.: I'm rea...,"[The, best, soundtrack, ever, anything, I, 'm,..."
2,2,Amazing!: This soundtrack is my favorite music...,"[Amazing, This, soundtrack, favorite, music, t..."
3,2,Excellent Soundtrack: I truly like this soundt...,"[Excellent, Soundtrack, I, truly, like, soundt..."
4,2,"Remember, Pull Your Jaw Off The Floor After He...","[Remember, Pull, Your, Jaw, Off, The, Floor, A..."
...,...,...,...
9995,2,A revelation of life in small town America in ...,"[A, revelation, life, small, town, America, ea..."
9996,2,Great biography of a very interesting journali...,"[Great, biography, interesting, journalist, Th..."
9997,1,Interesting Subject; Poor Presentation: You'd ...,"[Interesting, Subject, Poor, Presentation, You..."
9998,1,Don't buy: The box looked used and it is obvio...,"[Do, n't, buy, The, box, looked, used, obvious..."


In [65]:
lemmatizer = nltk.stem.wordnet.WordNetLemmatizer()
df['text_tokens'] = df['text_tokens'].apply(lambda l: [ lemmatizer.lemmatize(t) for t in l ])
df

,category,text,text_tokens
0,2,Stuning even for the non-gamer: This sound tra...,"[Stuning, even, non-gamer, This, sound, track,..."
1,2,The best soundtrack ever to anything.: I'm rea...,"[The, best, soundtrack, ever, anything, I, 'm,..."
2,2,Amazing!: This soundtrack is my favorite music...,"[Amazing, This, soundtrack, favorite, music, t..."
3,2,Excellent Soundtrack: I truly like this soundt...,"[Excellent, Soundtrack, I, truly, like, soundt..."
4,2,"Remember, Pull Your Jaw Off The Floor After He...","[Remember, Pull, Your, Jaw, Off, The, Floor, A..."
...,...,...,...
9995,2,A revelation of life in small town America in ...,"[A, revelation, life, small, town, America, ea..."
9996,2,Great biography of a very interesting journali...,"[Great, biography, interesting, journalist, Th..."
9997,1,Interesting Subject; Poor Presentation: You'd ...,"[Interesting, Subject, Poor, Presentation, You..."
9998,1,Don't buy: The box looked used and it is obvio...,"[Do, n't, buy, The, box, looked, used, obvious..."


In [90]:
from collections import Counter
corpus = [ t for ts in df.text_tokens for t in ts ]
freq_dicts = list(tp[0] for tp in sorted(Counter(corpus).items(), key=lambda x: -x[1]))

freq_dicts = { 
    'all': set(freq_dicts),
    '<5%': set(freq_dicts[:len(freq_dicts)//20]), 
    '5-20%': set(freq_dicts[len(freq_dicts)//20 : len(freq_dicts)//5]),  
    '>20%': set(freq_dicts[len(freq_dicts)//5:]),
}


In [119]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer


df_train, df_test = train_test_split(df, test_size=0.2, random_state=42)

def filtered_text(df_, freq_type):
    return [ ' '.join(t for t in tt if t in freq_dicts[freq_type]) for tt in df_['text_tokens'] ]
    
def fit_and_test(freq_type, vec=None, clf=None):
    print('Символы частоты: ' + freq_type)
    x_train = filtered_text(df_train, freq_type)
    y_train = df_train['category']
    x_test = filtered_text(df_test, freq_type)
    y_test = df_test['category']
    
    if clf == None:
        clf = LogisticRegression(random_state=42)

    bow = vec.fit_transform(x_train)
    clf.fit(bow, y_train)
    
    
    if vec is not None and hasattr(vec, 'get_feature_names'):
        feature_names = vec.get_feature_names()
        coefs_with_importances = sorted(zip(clf.coef_[0], feature_names))
        n_important = 10;
        print("Наиболее значимые положительные: ")
        for t in reversed(coefs_with_importances[-n_important:]):
            print(f"{t[1]} : {t[0]:.3f}")
        print("Наиболее значимые отрицательные: ")
        for t in coefs_with_importances[:n_important]:
            print(f"{t[1]} : {t[0]:.3f}")

    pred = clf.predict(vec.transform(x_test))
    print(classification_report(pred, y_test))
    


In [ ]:
print('CountVectorizer:')
for freq_type in freq_dicts:
    fit_and_test(freq_type, CountVectorizer(ngram_range=(1, 1)))
    
print('TfidfVectorizer:')
for freq_type in freq_dicts:
    fit_and_test(freq_type, TfidfVectorizer(ngram_range=(1, 1)))
    
for n_features in [100, 200, 500, 1000, 10000, 100000, 1000000]:
    print(f'HashingVectorizer with {n_features} features:')
    fit_and_test('all', HashingVectorizer(analyzer='word', n_features=n_features))


In [139]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization 

def build_nn_vectorizer(n_features, freq_type='all'):
    vectorize_layer = TextVectorization(max_tokens=n_features, output_mode='int', output_sequence_length=100)
    vectorize_layer.adapt(filtered_text(df, freq_type))
    model = Sequential([
        tf.keras.Input(shape=(1,), dtype=tf.string),
        vectorize_layer,
#        Embedding(n_features, 200),
#        Dense(1)
    ])
    model.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy())
    return model


x_train = filtered_text(df_train, freq_type)
y_train = df_train['category'].tolist()
x_test = filtered_text(df_test, freq_type)
y_test = df_test['category'].tolist()

clf = build_nn_vectorizer(10000)

print(x_train[:3])

clf.fit(x_train, y_train)



['Nutritious freshest available.What preperation fore pea leftover recipes.One Vegan dairy eggs.Overall', 'pneumatic etch', 'Elegance']


TypeError: in user code:

    C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:805 train_function  *
        return step_function(self, iterator)
    C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:795 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:3417 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:788 run_step  **
  

### Выводы по заданию 1:
Наилучший результат получился с использованием наиболее популярных токенов, как с CountVectorizer, так и с TfidfVectorizer. Вероятно, наименее популярные токены хорошо выделяют отдельные документы, но не категорию. Самый лучший результат получался по полному набору токенов

### Выводы по заданию 2:
Фичи с наибольшей значимостью вполне подходят к каждой из категорий, особенно для полного и наиболее популярного наборов

### Выводы по заданию 3:
1. (кроме полносвязной сетки) - лучше всего отработал TfIdfVectorizer. HashingVectorizer приблизился к этому результату на больших размерах > 10000
2. Близкие к максимальным метрики получаются у HashingVectorizer, начиная с размера 10000, но продолжают слегка улучшаться вплоть до 1000000.